In [1]:
from gurobipy import *
import csv

In [2]:
data = {}
posset = set()
with open('final.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='"')
    i = 0
    for row in spamreader:
        if i != 0:
            data[row[0]] = [row[1].split('-'), float(row[2][1:].strip().replace(',', '')), float(row[3].strip().replace(',', ''))]
            for pos in row[1].split('-'):
                posset.add(pos)
        i += 1
pos_set = list(posset)
print(pos_set)

['SG', 'C', 'SF', 'PF', 'PG']


In [3]:
for key in data.keys():
    print(key,data[key])

Aaron Brooks [['PG'], 2700000.0, 1361.4]
Aaron Gordon [['PF'], 4351320.0, 926.0]
Aaron Gray [['C'], 452059.0, 356.25]
Aaron Harrison [['SG'], 375579.0, 37.0]
Adreian Payne [['PF'], 2022240.0, 520.5]
Al Horford [['C'], 26540100.0, 1631.166667]
Al Jefferson [['C'], 10230179.0, 2036.333333]
Al-Farouq Aminu [['SF'], 7680965.0, 1065.833333]
Alan Anderson [['SF'], 1315448.0, 576.8]
Alan Williams [['PF'], 874636.0, 75.0]
Alec Burks [['SG'], 10154495.0, 768.4]
Alex Len [['C'], 4823621.0, 839.0]
Alexis Ajinca [['C'], 4600000.0, 649.5]
Allen Crabbe [['SG'], 18500000.0, 503.0]
Alonzo Gee [['SG'], 77379.0, 893.0]
Amir Johnson [['PF'], 12000000.0, 1305.0]
Anderson Varejao [['C'], 1551659.0, 752.1666667]
Andre Drummond [['C'], 22116750.0, 2108.25]
Andre Iguodala [['SF'], 11131368.0, 1395.166667]
Andre Roberson [['SG'], 2183072.0, 492.6666667]
Andrea Bargnani [['C'], 323599.0, 838.6666667]
Andrew Bogut [['C'], 11027027.0, 1036.333333]
Andrew Nicholson [['PF'], 6088993.0, 616.0]
Andrew Wiggins [['SF']

In [4]:
#Intialize solver module
m = Model("mip1")

In [5]:
#Creating ILP variables
play={}
for key in data.keys():
    x = m.addVar(vtype=GRB.BINARY, name=key)
    play[key] = x

In [6]:
#Setting ILP objective
objective = 0
for key in data.keys():
    objective += play[key] * data[key][2]
m.setObjective(objective, GRB.MAXIMIZE)

In [7]:
#Adding position constraints
for p in posset:
    p_constr = 0
    for key in data.keys():
        if p in data[key][0]:
            v = 1
        else:
            v = 0
        p_constr += play[key] * v
    m.addConstr(p_constr >= 2, p + "1")
    m.addConstr(p_constr <= 2, p + "2")

In [8]:
#Adding salary constraint
s_constr = 0
for key in data.keys():
    s_constr += play[key] * data[key][1]
m.addConstr(s_constr <= 100000000 , "salary constraint")

<gurobi.Constr *Awaiting Model Update*>

In [9]:
m.setParam('OutputFlag', False)
m.optimize()

In [10]:
i = 0
cost = 0
for ilp_variable in m.getVars():
    if ilp_variable.x == 1:
        print(i, ilp_variable.varName)
        cost += data[ilp_variable.varName][1]
        i += 1
perf = m.objVal
print(perf)
print(cost)

0 David Lee
1 Deron Williams
2 James Harden
3 Josh Smith
4 Karl-Anthony Towns
5 LeBron James
6 Michael Carter-Williams
7 Monta Ellis
8 Rudy Gay
9 Tim Duncan
22862.333334
99843104.0
